In [42]:
import pandas as pd
pd.set_option('display.max_columns', None)
import json
import numpy as np
from scipy.sparse import save_npz, csc_matrix
from sklearn.preprocessing import OneHotEncoder
#from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm
tqdm.pandas()

### Смотрим на данные

In [2]:
data1 = pd.read_csv('lst_announcement_data_1.gzip', compression='gzip')
data2 = pd.read_csv('lst_announcement_data_2.gzip', compression='gzip')

In [3]:
print(data1.shape)
data1.head()

(1000000, 35)


,id,floornumber,category,bargainterms,description,totalarea,geo,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasgarage,hasshower,bedroomscount,hasdishwasher,repairtype,petsallowed,hasbathhouse,hasfridge,wclocationtype,kitchenarea,roomarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,ptn_dadd
0,243455495,NaN,garageSale,"{""price"":3700000.0,""currency"":""rur"",""priceType...",В районе улицы Старый Арбат в клубном доме пре...,12.5,"{""countryId"":138,""undergrounds"":[{""transportTy...","{""parking"":{""type"":""underground""},""cranageType...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-26
1,270527203,2.0,dailyRoomRent,"{""price"":500.0,""currency"":""rur"",""deposit"":0,""b...",Сдается посуточно комфортабельный номер (18м2)...,NaN,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""monolithBrick"",""parking"":{""ty...",{},True,6.0,True,NaN,True,NaN,NaN,NaN,True,NaN,NaN,design,NaN,NaN,True,NaN,NaN,18.0,NaN,NaN,1.0,NaN,NaN,rooms,1.0,2022-06-26
2,270569384,7.0,flatRent,"{""clientFee"":35,""leaseTermType"":""longTerm"",""pr...",Сдам квартиру в хорошем состоянии,38.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""cargoLiftsCount"":0,""materialType"":""panel"",""h...",{},True,1.0,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN,cosmetic,False,NaN,True,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,2022-06-26
3,271459928,4.0,flatRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...",Сдам новую 1-комнатную квартиру в Болховском м...,42.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""name"":""ЖК «Мкр. Болховский»"",""parking"":{},""f...",{},NaN,1.0,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,cosmetic,NaN,NaN,True,NaN,NaN,NaN,False,NaN,1.0,NaN,NaN,rooms,NaN,2022-06-26
4,272357233,NaN,houseSale,"{""price"":3500000.0,""currency"":""rur"",""priceType...","Продам дачу в снт Тепличный-1, участок площадь...",77.1,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""cranageTypes"":[],""extinguishing...","{""areaUnitType"":""sotka"",""area"":6.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-26


In [4]:
print(data2.shape)
data2.head()

(963100, 35)


,id,floornumber,category,bargainterms,description,totalarea,geo,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasgarage,hasshower,bedroomscount,hasdishwasher,repairtype,petsallowed,hasbathhouse,hasfridge,wclocationtype,kitchenarea,roomarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,ptn_dadd
0,275092765,3.0,officeRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...","Описание Бизнес-центра: Деловой центр ""Салова ...",54.5,"{""countryId"":138,""undergrounds"":[{""transportTy...","{""name"":""Салова 45"",""parking"":{""priceMonthly"":...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-25
1,275118966,3.0,flatRent,"{""clientFee"":50,""leaseTermType"":""longTerm"",""pr...",Сдаётся светлая просторная 2-к квартира на дли...,45.0,"{""countryId"":138,""undergrounds"":[{""transportTy...","{""materialType"":""brick"",""hasGarbageChute"":fals...",{},True,2.0,False,True,True,True,True,NaN,False,NaN,False,euro,True,NaN,True,NaN,6.0,NaN,False,NaN,1.0,NaN,25.0,rooms,NaN,2022-06-25
2,206591333,NaN,houseSale,"{""price"":250000.0,""currency"":""rur"",""priceType""...",Дом находится в селе Поника в 7 км. от посёлка...,45.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""wood"",""parking"":{},""floorsCou...","{""status"":""privateFarm"",""areaUnitType"":""sotka""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outdoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-25
3,240796058,NaN,houseSale,"{""price"":3450000.0,""currency"":""rur"",""priceType...",Продам земельный участок с домом в Советском р...,64.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""wood"",""parking"":{},""floorsCou...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-25
4,268872358,3.0,flatSale,"{""price"":2650000.0,""currency"":""rur"",""priceType...",Продается двухкомнатная квартира в Приокском!\...,44.8,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""brick"",""parking"":{},""floorsCo...",{},NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,design,NaN,NaN,NaN,NaN,6.5,NaN,False,NaN,1.0,NaN,30.0,rooms,1.0,2022-06-25


##### Удалим дубликаты (они есть!)

In [5]:
data1.drop_duplicates(subset=['id'], inplace=True)
data2.drop_duplicates(subset=['id'], inplace=True)
print(data1.shape, data2.shape)

(691120, 35) (679606, 35)


In [6]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 691120 entries, 0 to 999999
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   691120 non-null  int64  
 1   floornumber          484910 non-null  float64
 2   category             691120 non-null  object 
 3   bargainterms         691120 non-null  object 
 4   description          691120 non-null  object 
 5   totalarea            641033 non-null  float64
 6   geo                  691120 non-null  object 
 7   building             691120 non-null  object 
 8   land                 691120 non-null  object 
 9   hasfurniture         135325 non-null  object 
 10  roomscount           418236 non-null  float64
 11  hasconditioner       72368 non-null   object 
 12  haskitchenfurniture  130212 non-null  object 
 13  hastv                109137 non-null  object 
 14  haswasher            131002 non-null  object 
 15  hasbathtub       

Удаляем столбцы, в которых более 90% пропусков

In [8]:
data = pd.concat([data1, data2]) # не тестовый вариант
#data = data1.iloc[:10000] # тестовый вариант для ускорения
data.drop_duplicates(subset=['id'], inplace=True)
ID = np.array(data['id']) # сохраним id объявок

def get_nan_cols(df, nan_percent=0.8):
    """
        функция для определения колонок, в которых много NaN
    """
    threshold = len(df.index) * nan_percent
    return [c for c in df.columns if df[c].isna().sum() >= threshold]

colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
data.drop(['ptn_dadd'], axis=1, inplace=True) # неинформативный признак даты, удаляем
data.tail()

,id,floornumber,category,bargainterms,description,totalarea,geo,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount
963093,272826700,NaN,landSale,"{""price"":1111111.0,""currency"":""rur"",""priceType...","Продается просторный участок 15 соток, в собст...",NaN,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""cranageTypes"":[],""extinguishing...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
963094,273370267,10.0,flatSale,"{""price"":3890000.0,""currency"":""rur"",""priceType...",Продается отличная трехкомнатная квартира!\nОд...,56.8,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""floorsCount"":15,""cranageTypes"":...",{},NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,cosmetic,NaN,NaN,NaN,25.3,NaN,0.0,1.0,NaN,NaN,rooms,0.0
963095,273970898,16.0,flatRent,"{""clientFee"":70,""leaseTermType"":""longTerm"",""pr...",Есть вся необходимая мебель и бытовая техника ...,60.0,"{""countryId"":138,""undergrounds"":[{""transportTy...","{""materialType"":""brick"",""parking"":{},""passenge...",{},True,2.0,NaN,True,NaN,True,NaN,NaN,euro,NaN,True,NaN,NaN,False,NaN,NaN,NaN,NaN,rooms,NaN
963097,275082384,10.0,flatSale,"{""price"":4700000.0,""currency"":""rur"",""priceType...",Для отдыха и сдачи готовый пассивный бизнес.\n...,42.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""monolith"",""parking"":{},""passe...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,design,NaN,NaN,NaN,NaN,True,NaN,1.0,NaN,38.0,studio,1.0
963099,216727837,NaN,garageSale,"{""price"":280000.0,""currency"":""rur"",""priceType""...","Продам гараж.\nВ гаражом кооперативе ""Мотор-82...",22.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""name"":""Мотор-82"",""parking"":{},""cranageTypes""...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# посмотрим что находится в geo
data['geo'].iloc[0]

'{"countryId":138,"undergrounds":[{"transportType":"walk","name":"Кропоткинская","lineColor":"CF0000","time":9,"lineId":10,"id":56,"isDefault":true},{"transportType":"walk","name":"Арбатская","lineColor":"03238B","time":10,"lineId":1,"id":8,"isDefault":false},{"transportType":"walk","name":"Смоленская","lineColor":"03238B","time":11,"lineId":1,"id":115,"isDefault":false}],"calculatedUndergrounds":[{"transportType":"transport","time":2,"distance":1004,"id":8},{"transportType":"walk","time":10,"distance":872,"id":8},{"transportType":"walk","time":9,"distance":789,"id":56},{"transportType":"transport","time":2,"distance":984,"id":56},{"transportType":"walk","time":11,"distance":966,"id":115},{"transportType":"transport","time":4,"distance":1532,"id":115}],"coordinates":{"lat":55.748665,"lng":37.59383},"highways":[],"railways":[{"id":693,"directionIds":[14],"distance":3.0,"time":6,"name":"Москва (Киевский вокзал)","travelType":"byCar"},{"id":693,"directionIds":[14],"distance":2.0,"time":30

### Преобразуем признак geo

In [11]:
data['countryId'] = data['geo'].progress_apply(lambda x: json.loads(x)['countryId'])
data['lat'] = data['geo'].progress_apply(lambda x: json.loads(x)['coordinates']['lat'])
data['lng'] = data['geo'].progress_apply(lambda x: json.loads(x)['coordinates']['lng'])
data.drop(['geo'], axis=1, inplace=True)
data.head()

100%|██████████████████████████████| 1029010/1029010 [00:32<00:00, 31561.24it/s]


,id,floornumber,category,bargainterms,description,totalarea,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng
0,243455495,NaN,garageSale,"{""price"":3700000.0,""currency"":""rur"",""priceType...",В районе улицы Старый Арбат в клубном доме пре...,12.5,"{""parking"":{""type"":""underground""},""cranageType...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830
1,270527203,2.0,dailyRoomRent,"{""price"":500.0,""currency"":""rur"",""deposit"":0,""b...",Сдается посуточно комфортабельный номер (18м2)...,NaN,"{""materialType"":""monolithBrick"",""parking"":{""ty...",{},True,6.0,True,NaN,True,NaN,NaN,NaN,design,NaN,True,NaN,NaN,NaN,NaN,1.0,NaN,NaN,rooms,1.0,138,45.042789,35.377426
2,270569384,7.0,flatRent,"{""clientFee"":35,""leaseTermType"":""longTerm"",""pr...",Сдам квартиру в хорошем состоянии,38.0,"{""cargoLiftsCount"":0,""materialType"":""panel"",""h...",{},True,1.0,NaN,True,NaN,True,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,138,52.022010,47.842987
3,271459928,4.0,flatRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...",Сдам новую 1-комнатную квартиру в Болховском м...,42.0,"{""name"":""ЖК «Мкр. Болховский»"",""parking"":{},""f...",{},NaN,1.0,NaN,NaN,NaN,True,NaN,NaN,cosmetic,NaN,True,NaN,NaN,False,NaN,1.0,NaN,NaN,rooms,NaN,138,52.997077,36.064897
4,272357233,NaN,houseSale,"{""price"":3500000.0,""currency"":""rur"",""priceType...","Продам дачу в снт Тепличный-1, участок площадь...",77.1,"{""parking"":{},""cranageTypes"":[],""extinguishing...","{""areaUnitType"":""sotka"",""area"":6.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,54.819368,73.548189


### Преобразуем признаки-словари bargainterms, building, land

Составим новые колонки из ключей словарей исходных колонок

In [12]:
data['bargainterms'] = data['bargainterms'].progress_apply(json.loads)
keys_bargainterms = data['bargainterms'].apply(lambda x: x.keys()).explode().unique()
bargainterms_df = pd.DataFrame(data['bargainterms'].values.tolist())

data['building'] = data['building'].progress_apply(json.loads)
keys_building = data['building'].apply(lambda x: x.keys()).explode().unique()
building_df = pd.DataFrame(data['building'].values.tolist())

data['land'] = data['land'].progress_apply(json.loads)
keys_land = data['land'].apply(lambda x: x.keys()).explode().unique()
land_df = pd.DataFrame(data['land'].values.tolist())

land_df.rename(columns={'type': 'land_type'}, inplace=True)
data = data.join(bargainterms_df).join(building_df).join(land_df)
print(data.shape)
data.head()

100%|█████████████████████████████| 1029010/1029010 [00:03<00:00, 296643.66it/s]


(1029010, 97)


,id,floornumber,category,bargainterms,description,totalarea,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng,price,currency,priceType,includedOptions,deposit,bargainAllowed,tenantsType,clientFee,leaseTermType,agentFee,prepayMonths,paymentPeriod,utilitiesTerms,mortgageAllowed,saleType,agentBonus,actionId,leaseType,hasGracePeriod,vatIncluded,vatType,minLeaseTerm,securityDeposit,contractType,vatPrice,bargainPrice,bargainConditions,priceForWorkplace,parking,cranageTypes,extinguishingSystemTypes,liftTypes,infrastructure,openingHours,deadline,materialType,floorsCount,ceilingHeight,cargoLiftsCount,hasGarbageChute,passengerLiftsCount,totalArea,buildYear,name,heatingType,type,statusType,series,extinguishingSystemType,classType,ventilationType,conditioningType,developer,houseLineType,gatesType,shoppingCenterScaleType,workingDaysType,tenants,accessType,columnGrid,managementCompany,areaUnitType,area,status,land_type,possibleToChangeStatus
0,243455495,NaN,garageSale,"{'price': 3700000.0, 'currency': 'rur', 'price...",В районе улицы Старый Арбат в клубном доме пре...,12.5,"{'parking': {'type': 'underground'}, 'cranageT...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830,3700000.0,rur,all,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'type': 'underground'},[],[],[],"{'hasEntryByPass': True, 'hasAutomaticGates': ...",{},{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,275092765,3.0,officeRent,"{'clientFee': 0, 'leaseTermType': 'longTerm', ...","Описание Бизнес-центра: Деловой центр ""Салова ...",54.5,"{'name': 'Салова 45', 'parking': {'priceMonthl...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,59.890537,30.380610,3700000.0,rur,all,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'type': 'underground'},[],[],[],"{'hasEntryByPass': True, 'hasAutomaticGates': ...",{},{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,270527203,2.0,dailyRoomRent,"{'price': 500.0, 'currency': 'rur', 'deposit':...",Сдается посуточно комфортабельный номер (18м2)...,NaN,"{'materialType': 'monolithBrick', 'parking': {...",{},True,6.0,True,NaN,True,NaN,NaN,NaN,design,NaN,True,NaN,NaN,NaN,NaN,1.0,NaN,NaN,rooms,1.0,138,45.042789,35.377426,500.0,rur,all,[],0.0,False,any,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'type': 'ground'},[],[],[],{},{},{},monolithBrick,4.0,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,270569384,7.0,flatRent,"{'clientFee': 35, 'leaseTermType': 'longTerm',...",Сдам квартиру в хорошем состоянии,38.0,"{'cargoLiftsCount': 0, 'materialType': 'panel'...",{},True,1.0,NaN,True,NaN,True,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,138,52.022010,47.842987,11000.0,rur,all,[],0.0,NaN,NaN,35.0,longTerm,0.0,1.0,monthly,"{'includedInPrice': False, 'price': 0.0, 'flow...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},[],[],[],{},{},{},panel,9.0,NaN,0.0,True,1.0,38.0,2004.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,206591333,NaN,houseSale,"{'price': 250000.0, 'currency': 'rur', 'priceT...",Дом находится в селе Поника в 7 км. от посёлка...,45.0,"{'materialType': 'wood', 'parking': {}, 'floor...","{'status': 'privateFarm', 'areaUnitType': 'sot...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outdoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,53.188924,47.152452,11000.0,rur,all,[],0.0,

In [13]:
# почистим колонки с большим кол-вом пропусков и удалим ненужные словарные
data.drop(['bargainterms', 'building', 'land'], axis=1, inplace=True)
colss = get_nan_cols(data, 0.9)
data.drop(colss, axis=1, inplace=True)
data.head()

,id,floornumber,category,description,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng,price,currency,priceType,includedOptions,deposit,clientFee,leaseTermType,agentFee,prepayMonths,paymentPeriod,utilitiesTerms,mortgageAllowed,saleType,agentBonus,parking,cranageTypes,extinguishingSystemTypes,liftTypes,infrastructure,openingHours,deadline,materialType,floorsCount,ceilingHeight,cargoLiftsCount,hasGarbageChute,passengerLiftsCount,totalArea,buildYear,name,heatingType,areaUnitType,area,status
0,243455495,NaN,garageSale,В районе улицы Старый Арбат в клубном доме пре...,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830,3700000.0,rur,all,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'type': 'underground'},[],[],[],"{'hasEntryByPass': True, 'hasAutomaticGates': ...",{},{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,275092765,3.0,officeRent,"Описание Бизнес-центра: Деловой центр ""Салова ...",54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,59.890537,30.380610,3700000.0,rur,all,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'type': 'underground'},[],[],[],"{'hasEntryByPass': True, 'hasAutomaticGates': ...",{},{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,270527203,2.0,dailyRoomRent,Сдается посуточно комфортабельный номер (18м2)...,NaN,True,6.0,NaN,True,NaN,NaN,NaN,design,NaN,True,NaN,NaN,NaN,NaN,1.0,NaN,NaN,rooms,1.0,138,45.042789,35.377426,500.0,rur,all,[],0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'type': 'ground'},[],[],[],{},{},{},monolithBrick,4.0,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,270569384,7.0,flatRent,Сдам квартиру в хорошем состоянии,38.0,True,1.0,True,NaN,True,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,138,52.022010,47.842987,11000.0,rur,all,[],0.0,35.0,longTerm,0.0,1.0,monthly,"{'includedInPrice': False, 'price': 0.0, 'flow...",NaN,NaN,NaN,{},[],[],[],{},{},{},panel,9.0,NaN,0.0,True,1.0,38.0,2004.0,NaN,NaN,NaN,NaN,NaN
2,206591333,NaN,houseSale,Дом находится в селе Поника в 7 км. от посёлка...,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outdoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,53.188924,47.152452,11000.0,rur,all,[],0.0,35.0,longTerm,0.0,1.0,monthly,"{'includedInPrice': False, 'price': 0.0, 'flow...",NaN,NaN,NaN,{},[],[],[],{},{},{},panel,9.0,NaN,0.0,True,1.0,38.0,2004.0,NaN,NaN,NaN,NaN,NaN


##### Оставшиеся колонки типа list: 
includedOptions, cranageTypes, extinguishingSystemTypes, liftTypes	

##### Оставшиеся колонки типа dict: 
utilitiesTerms, parking, infrastructure, openingHours, deadline, agentBonus

##### Тип пока непонятен для колонок: 
bedroomscount, wclocationtype, allroomsarea, name, heatingType, areaUnitType, area, status

In [14]:
# посмотрим, сколько пустых словарей/списков
data['utilitiesTerms'] = data['utilitiesTerms'].progress_apply(lambda y: np.nan if y is None else y)
data['parking'] = data['parking'].progress_apply(lambda y: np.nan if len(y)==0 else y)
data['infrastructure'] = data['infrastructure'].progress_apply(lambda y: np.nan if len(y)==0 else y)
data['openingHours'] = data['openingHours'].progress_apply(lambda y: np.nan if len(y)==0 else y)
data['deadline'] = data['deadline'].progress_apply(lambda y: np.nan if len(y)==0 else y)
data['agentBonus'] = data['agentBonus'].progress_apply(lambda y: np.nan if y is None else y)

data['includedOptions'] = data['includedOptions'].progress_apply(lambda y: np.nan if y is None else y)
data['cranageTypes'] = data['cranageTypes'].progress_apply(lambda y: np.nan if y is None else y)
data['extinguishingSystemTypes'] = data['extinguishingSystemTypes'].progress_apply(lambda y: np.nan if y is None else y)
data['liftTypes'] = data['liftTypes'].progress_apply(lambda y: np.nan if y is None else y)
data.head()

100%|█████████████████████████████| 1029010/1029010 [00:01<00:00, 863543.64it/s]


,id,floornumber,category,description,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng,price,currency,priceType,includedOptions,deposit,clientFee,leaseTermType,agentFee,prepayMonths,paymentPeriod,utilitiesTerms,mortgageAllowed,saleType,agentBonus,parking,cranageTypes,extinguishingSystemTypes,liftTypes,infrastructure,openingHours,deadline,materialType,floorsCount,ceilingHeight,cargoLiftsCount,hasGarbageChute,passengerLiftsCount,totalArea,buildYear,name,heatingType,areaUnitType,area,status
0,243455495,NaN,garageSale,В районе улицы Старый Арбат в клубном доме пре...,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830,3700000.0,rur,all,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'type': 'underground'},[],[],[],"{'hasEntryByPass': True, 'hasAutomaticGates': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,275092765,3.0,officeRent,"Описание Бизнес-центра: Деловой центр ""Салова ...",54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,59.890537,30.380610,3700000.0,rur,all,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'type': 'underground'},[],[],[],"{'hasEntryByPass': True, 'hasAutomaticGates': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,270527203,2.0,dailyRoomRent,Сдается посуточно комфортабельный номер (18м2)...,NaN,True,6.0,NaN,True,NaN,NaN,NaN,design,NaN,True,NaN,NaN,NaN,NaN,1.0,NaN,NaN,rooms,1.0,138,45.042789,35.377426,500.0,rur,all,[],0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'type': 'ground'},[],[],[],NaN,NaN,NaN,monolithBrick,4.0,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,270569384,7.0,flatRent,Сдам квартиру в хорошем состоянии,38.0,True,1.0,True,NaN,True,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,138,52.022010,47.842987,11000.0,rur,all,[],0.0,35.0,longTerm,0.0,1.0,monthly,"{'includedInPrice': False, 'price': 0.0, 'flow...",NaN,NaN,NaN,NaN,[],[],[],NaN,NaN,NaN,panel,9.0,NaN,0.0,True,1.0,38.0,2004.0,NaN,NaN,NaN,NaN,NaN
2,206591333,NaN,houseSale,Дом находится в селе Поника в 7 км. от посёлка...,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outdoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,53.188924,47.152452,11000.0,rur,all,[],0.0,35.0,longTerm,0.0,1.0,monthly,"{'includedInPrice': False, 'price': 0.0, 'flow...",NaN,NaN,NaN,NaN,[],[],[],NaN,NaN,NaN,panel,9.0,NaN,0.0,True,1.0,38.0,2004.0,NaN,NaN,NaN,NaN,NaN


In [15]:
# традиционно почистим от колонок, в которых много пропусков
print(data.shape)
colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
print(data.shape)

(1029010, 61)
(1029010, 58)


##### Странные значения в колонках descriptions, allroomsarea, includedOptions, agentBonus, name
##### И в числовых cranageTypes, extinguishSystemTypes, liftTypes
удалим их

In [16]:
data.drop(['description', 'allroomsarea', 'agentBonus', 'name'], axis=1, inplace=True) # 'includedOptions' уже пропала
data.drop('liftTypes', axis=1, inplace=True) # 'cranageTypes' 'extinguishSystemTypes' тоже

In [17]:
# нужно переименовать ключ price в utilitiesTerms и type в parking
def rename_price(arg):
    if arg is np.nan:
        return arg
    try:
        arg['comm_price'] = arg['price']
        arg.pop('price')
        return arg
    except:
        pass
data['utilitiesTerms'].apply(rename_price)

def rename_type(arg):
    if arg is np.nan:
        return arg
    try:
        arg['parking_type'] = arg['type']
        arg.pop('type')
        return arg
    except:
        pass
data['parking'].apply(rename_type)

0         {'parking_type': 'underground'}
0                                    None
1              {'parking_type': 'ground'}
2                                     NaN
2                                     NaN
                       ...               
999993                                NaN
999994                                NaN
999996                                NaN
999997                                NaN
999999                                NaN
Name: parking, Length: 1029010, dtype: object

In [18]:
##### for i in tqdm(list(...)):

In [19]:
print(data['utilitiesTerms'].iloc[2])
print(data['parking'].iloc[2])

nan
{'parking_type': 'ground'}


##### ещё словарные признаки - utilitiesTerms, parking - распакуем их

In [20]:
data[['parking', 'utilitiesTerms']] = data[['parking', 'utilitiesTerms']].applymap(lambda x: \
                                                                                   {} if pd.isnull(x) else x)
parking_df = pd.DataFrame(data['parking'].values.tolist())
utilitiesTerms_df = pd.DataFrame(data['utilitiesTerms'].dropna().values.tolist())

parking_df.rename(columns={'currency': 'curr'}, inplace=True)
data = data.join(parking_df).join(utilitiesTerms_df)

In [21]:
data.head()

,id,floornumber,category,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,livingarea,flattype,combinedwcscount,countryId,lat,lng,price,currency,priceType,includedOptions,deposit,clientFee,leaseTermType,agentFee,prepayMonths,paymentPeriod,utilitiesTerms,mortgageAllowed,saleType,parking,cranageTypes,extinguishingSystemTypes,materialType,floorsCount,ceilingHeight,cargoLiftsCount,hasGarbageChute,passengerLiftsCount,totalArea,buildYear,heatingType,areaUnitType,area,status,parking_type,isFree,curr,locationType,purposeType,placesCount,priceMonthly,priceEntry,includedInPrice,flowMetersNotIncludedInPrice,comm_price
0,243455495,NaN,garageSale,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830,3700000.0,rur,all,[],NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,{'parking_type': 'underground'},[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,275092765,3.0,officeRent,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,59.890537,30.380610,3700000.0,rur,all,[],NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,{'parking_type': 'underground'},[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,270527203,2.0,dailyRoomRent,NaN,True,6.0,NaN,True,NaN,NaN,NaN,design,NaN,True,NaN,NaN,NaN,NaN,1.0,NaN,rooms,1.0,138,45.042789,35.377426,500.0,rur,all,[],0.0,NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,{'parking_type': 'ground'},[],[],monolithBrick,4.0,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,270569384,7.0,flatRent,38.0,True,1.0,True,NaN,True,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,0.0,1.0,NaN,rooms,0.0,138,52.022010,47.842987,11000.0,rur,all,[],0.0,35.0,longTerm,0.0,1.0,monthly,"{'includedInPrice': False, 'flowMetersNotInclu...",NaN,NaN,{},[],[],panel,9.0,NaN,0.0,True,1.0,38.0,2004.0,NaN,NaN,NaN,NaN,ground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,206591333,NaN,houseSale,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outdoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,53.188924,47.152452,11000.0,rur,all,[],0.0,35.0,longTerm,0.0,1.0,monthly,"{'includedInPrice': False, 'flowMetersNotInclu...",NaN,NaN,{},[],[],panel,9.0,NaN,0.0,True,1.0,38.0,2004.0,NaN,NaN,NaN,NaN,ground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
data = data.drop(['parking', 'utilitiesTerms'], axis=1)

In [23]:
colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
data.head()

,id,floornumber,category,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,livingarea,flattype,combinedwcscount,countryId,lat,lng,price,currency,priceType,includedOptions,deposit,clientFee,leaseTermType,agentFee,prepayMonths,paymentPeriod,mortgageAllowed,saleType,cranageTypes,extinguishingSystemTypes,materialType,floorsCount,ceilingHeight,cargoLiftsCount,hasGarbageChute,passengerLiftsCount,totalArea,buildYear,heatingType,areaUnitType,area,status,parking_type,includedInPrice,comm_price
0,243455495,NaN,garageSale,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830,3700000.0,rur,all,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underground,NaN,NaN
0,275092765,3.0,officeRent,54.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,59.890537,30.380610,3700000.0,rur,all,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underground,NaN,NaN
1,270527203,2.0,dailyRoomRent,NaN,True,6.0,NaN,True,NaN,NaN,NaN,design,NaN,True,NaN,NaN,NaN,NaN,1.0,NaN,rooms,1.0,138,45.042789,35.377426,500.0,rur,all,[],0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[],monolithBrick,4.0,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underground,NaN,NaN
2,270569384,7.0,flatRent,38.0,True,1.0,True,NaN,True,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,0.0,1.0,NaN,rooms,0.0,138,52.022010,47.842987,11000.0,rur,all,[],0.0,35.0,longTerm,0.0,1.0,monthly,NaN,NaN,[],[],panel,9.0,NaN,0.0,True,1.0,38.0,2004.0,NaN,NaN,NaN,NaN,ground,NaN,NaN
2,206591333,NaN,houseSale,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outdoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,53.188924,47.152452,11000.0,rur,all,[],0.0,35.0,longTerm,0.0,1.0,monthly,NaN,NaN,[],[],panel,9.0,NaN,0.0,True,1.0,38.0,2004.0,NaN,NaN,NaN,NaN,ground,NaN,NaN


### Преобразование категориальных признаков

In [24]:
# выделим категориальные и числовые признаки
types = data.dtypes
numerical_cols = []
cat_cols = []
for i in tqdm(range(len(types))):
    if types[i] != 'object':
        numerical_cols.append(data.columns[i])
    else:
        cat_cols.append(data.columns[i])
#print(types)
#print(numerical_cols)
#print(cat_cols)

100%|███████████████████████████████████████| 54/54 [00:00<00:00, 196949.93it/s]


In [25]:
data[cat_cols].head()

,category,hasfurniture,haskitchenfurniture,hastv,haswasher,hasbathtub,repairtype,petsallowed,hasfridge,wclocationtype,isapartments,flattype,currency,priceType,includedOptions,leaseTermType,paymentPeriod,mortgageAllowed,saleType,cranageTypes,extinguishingSystemTypes,materialType,hasGarbageChute,heatingType,areaUnitType,status,parking_type,includedInPrice
0,garageSale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rur,all,[],NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN,NaN,underground,NaN
0,officeRent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rur,all,[],NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN,NaN,underground,NaN
1,dailyRoomRent,True,NaN,True,NaN,NaN,design,NaN,True,NaN,NaN,rooms,rur,all,[],NaN,NaN,NaN,NaN,[],[],monolithBrick,NaN,NaN,NaN,NaN,underground,NaN
2,flatRent,True,True,NaN,True,NaN,cosmetic,False,True,NaN,NaN,rooms,rur,all,[],longTerm,monthly,NaN,NaN,[],[],panel,True,NaN,NaN,NaN,ground,NaN
2,houseSale,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outdoors,NaN,NaN,rur,all,[],longTerm,monthly,NaN,NaN,[],[],panel,True,NaN,NaN,NaN,ground,NaN


In [35]:
numerical_cols.pop(0)

'id'

In [36]:
data[numerical_cols].head()

,floornumber,totalarea,roomscount,bedroomscount,kitchenarea,loggiascount,balconiescount,livingarea,combinedwcscount,countryId,lat,lng,price,deposit,clientFee,agentFee,prepayMonths,floorsCount,ceilingHeight,cargoLiftsCount,passengerLiftsCount,totalArea,buildYear,area,comm_price
0,6.178377,12.500000,2.00529,3.243381,10.789058,0.542892,0.635489,31.646817,0.744954,138,55.748665,37.593830,3700000.0,609013.434003,30.378807,26.060979,1.035548,9.828237,2.943148,0.646668,1.095944,375.101136,1993.231653,280.324923,712.656646
0,3.000000,54.500000,2.00529,3.243381,10.789058,0.542892,0.635489,31.646817,0.744954,138,59.890537,30.380610,3700000.0,609013.434003,30.378807,26.060979,1.035548,9.828237,2.943148,0.646668,1.095944,375.101136,1993.231653,280.324923,712.656646
1,2.000000,161.420441,6.00000,3.243381,10.789058,0.542892,1.000000,31.646817,1.000000,138,45.042789,35.377426,500.0,0.000000,30.378807,26.060979,1.035548,4.000000,2.800000,0.646668,1.095944,375.101136,1993.231653,280.324923,712.656646
2,7.000000,38.000000,1.00000,3.243381,10.789058,0.000000,1.000000,31.646817,0.000000,138,52.022010,47.842987,11000.0,0.000000,35.000000,0.000000,1.000000,9.000000,2.943148,0.000000,1.000000,38.000000,2004.000000,280.324923,712.656646
2,6.178377,45.000000,2.00529,3.243381,10.789058,0.542892,0.635489,31.646817,0.744954,138,53.188924,47.152452,11000.0,0.000000,35.000000,0.000000,1.000000,9.000000,2.943148,0.000000,1.000000,38.000000,2004.000000,280.324923,712.656646


In [37]:
# обработка пропусков
# для числовых признаков
def fillna_num(data_col):
    return data_col.fillna(data_col.mean())

# для категориальных
def fillna_cat(data_col):
    return data_col.fillna('-1-')

for col in tqdm(numerical_cols):
    data[col] = fillna_num(data[col])
    
for col in tqdm(cat_cols):
    data[col] = fillna_cat(data[col])

data.head()

100%|███████████████████████████████████████████| 28/28 [00:05<00:00,  5.20it/s]


,id,floornumber,category,totalarea,hasfurniture,roomscount,haskitchenfurniture,hastv,haswasher,hasbathtub,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,livingarea,flattype,combinedwcscount,countryId,lat,lng,price,currency,priceType,includedOptions,deposit,clientFee,leaseTermType,agentFee,prepayMonths,paymentPeriod,mortgageAllowed,saleType,cranageTypes,extinguishingSystemTypes,materialType,floorsCount,ceilingHeight,cargoLiftsCount,hasGarbageChute,passengerLiftsCount,totalArea,buildYear,heatingType,areaUnitType,area,status,parking_type,includedInPrice,comm_price
0,243455495,6.178377,garageSale,12.500000,-1-,2.00529,-1-,-1-,-1-,-1-,3.243381,-1-,-1-,-1-,-1-,10.789058,-1-,0.542892,0.635489,31.646817,-1-,0.744954,138,55.748665,37.593830,3700000.0,rur,all,[],609013.434003,30.378807,-1-,26.060979,1.035548,-1-,-1-,-1-,[],[],-1-,9.828237,2.943148,0.646668,-1-,1.095944,375.101136,1993.231653,-1-,-1-,280.324923,-1-,underground,-1-,712.656646
0,275092765,3.000000,officeRent,54.500000,-1-,2.00529,-1-,-1-,-1-,-1-,3.243381,-1-,-1-,-1-,-1-,10.789058,-1-,0.542892,0.635489,31.646817,-1-,0.744954,138,59.890537,30.380610,3700000.0,rur,all,[],609013.434003,30.378807,-1-,26.060979,1.035548,-1-,-1-,-1-,[],[],-1-,9.828237,2.943148,0.646668,-1-,1.095944,375.101136,1993.231653,-1-,-1-,280.324923,-1-,underground,-1-,712.656646
1,270527203,2.000000,dailyRoomRent,161.420441,True,6.00000,-1-,True,-1-,-1-,3.243381,design,-1-,True,-1-,10.789058,-1-,0.542892,1.000000,31.646817,rooms,1.000000,138,45.042789,35.377426,500.0,rur,all,[],0.000000,30.378807,-1-,26.060979,1.035548,-1-,-1-,-1-,[],[],monolithBrick,4.000000,2.800000,0.646668,-1-,1.095944,375.101136,1993.231653,-1-,-1-,280.324923,-1-,underground,-1-,712.656646
2,270569384,7.000000,flatRent,38.000000,True,1.00000,True,-1-,True,-1-,3.243381,cosmetic,False,True,-1-,10.789058,-1-,0.000000,1.000000,31.646817,rooms,0.000000,138,52.022010,47.842987,11000.0,rur,all,[],0.000000,35.000000,longTerm,0.000000,1.000000,monthly,-1-,-1-,[],[],panel,9.000000,2.943148,0.000000,True,1.000000,38.000000,2004.000000,-1-,-1-,280.324923,-1-,ground,-1-,712.656646
2,206591333,6.178377,houseSale,45.000000,-1-,2.00529,-1-,-1-,-1-,-1-,3.243381,-1-,-1-,-1-,outdoors,10.789058,-1-,0.542892,0.635489,31.646817,-1-,0.744954,138,53.188924,47.152452,11000.0,rur,all,[],0.000000,35.000000,longTerm,0.000000,1.000000,monthly,-1-,-1-,[],[],panel,9.000000,2.943148,0.000000,True,1.000000,38.000000,2004.000000,-1-,-1-,280.324923,-1-,ground,-1-,712.656646


In [28]:
# кодирование категориальных признаков
data_cat = data[cat_cols]
enc = OneHotEncoder(handle_unknown='ignore')
cat_enc = enc.fit_transform(data_cat.astype(str)).toarray()
display(cat_enc)
print(cat_enc.shape)

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

(1029010, 212)


In [29]:
# обработка description
#vectorizer = TfidfVectorizer(max_features=5000)
#descr_vect = vectorizer.fit_transform(data['description']).toarray()
#print(descr_vect.shape)
#display(descr_vect)

In [30]:
#vectorizer.vocabulary_

In [47]:
# отмасштабируем числовые признаки, кроме id
num_cols = data[numerical_cols]
#num_cols = num_cols.drop(['id'], axis=1)
scaler = MinMaxScaler()
num_cols = scaler.fit_transform(num_cols)
print(np.max(num_cols), np.min(num_cols))

1.0000000000000002 0.0


In [60]:
#embeddings = np.hstack((num_cols, cat_enc))
embeddings = np.hstack((ID.reshape((len(ID), 1)), num_cols, cat_enc))
print(embeddings.shape)
embeddings

(1029010, 238)


array([[2.43455495e+08, 7.28442599e-02, 1.66666691e-06, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.75092765e+08, 4.76190476e-02, 7.75362431e-06, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.70527203e+08, 3.96825397e-02, 2.32493426e-05, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.74939225e+08, 1.03174603e-01, 4.73913112e-06, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.74969832e+08, 2.30158730e-01, 1.28985526e-05, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.75026673e+08, 3.96825397e-02, 4.49275427e-06, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [51]:
ID # сохранённый список id

array([243455495, 275092765, 270527203, ..., 274939225, 274969832,
       275026673])

In [52]:
type(embeddings)

numpy.ndarray

### Сохранение в разных форматах

In [53]:
# save as .txt
#np.savetxt('embeddings_test10000.txt', embeddings, delimiter = ' ')

In [54]:
# save as sparse matrix
sparse_embeddings = csc_matrix(embeddings)
save_npz('embeddings_test10000.npz', sparse_embeddings)

In [55]:
# save in binary format
np.save('embeddings_ALL.npy', embeddings)  

In [62]:
# save ID
np.save('embeddings_ID.npy', embeddings[:, 0])  

In [56]:
# make dict
embeddings = np.hstack((num_cols, cat_enc))
print(ID.shape, type(ID))
print(embeddings.shape, type(embeddings))
embeddingsID = {}
for i in range(len(ID)):
    embeddingsID[ID[i]] = embeddings[i]
#print(embeddingsID)

(1029010,) <class 'numpy.ndarray'>
(1029010, 237) <class 'numpy.ndarray'>
